In [ ]:
!pip install pandas
!pip install timm
!pip install --upgrade timm

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
!mkdir jigsaw
!unzip -qq "/content/drive/MyDrive/open.zip" -d /content/jigsaw

mkdir: cannot create directory ‘DATA’: File exists
replace /content/DATA/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [9]:
import pandas as pd
df =pd.read_csv('train.csv')
print(df.iloc[13:16])
row = df.iloc[3]
# image = read_image(os.path.join(self.data_path, row['img_path']))
#train.csv의 jigsaw puzzle 풀기전 order
preseq = row[[str(i) for i in range(1, 17)]].values-1

             ID                 img_path   1   2   3   4   5   6   7   8   9  \
13  TRAIN_00013  ./train/TRAIN_00013.jpg  10   4  11  15   3   9  12   1  16   
14  TRAIN_00014  ./train/TRAIN_00014.jpg   3   2  16   1  12   6  14  11   8   
15  TRAIN_00015  ./train/TRAIN_00015.jpg  15  14  10  13   6  11   3   2  12   

    10  11  12  13  14  15  16  
13   8   6  14   7  13   5   2  
14   5   9   4  10  15   7  13  
15   9   4   7   5   1  16   8  


In [5]:
shuffle_order

array([13, 0, 5, 14, 8, 3, 7, 4, 15, 9, 12, 6, 1, 2, 10, 11], dtype=object)

In [29]:
from PIL import Image
image_path = "DATA/train/TRAIN_00000.jpg"
image = Image.open(image_path)
width, height =image.size
print(width,height)  #512,512 pixel 사이즈

512 512


In [ ]:
import timm
from timm.data import create_transform
from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
class model(nn.Module):
  deit = timm.create_model('deit_base_distilled_patch16_384', pretrained = True)
  # teacher-student strategy 학생이 선생님의 attention 학습
  # distilled patch embedding Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  pass



In [42]:
import torch
import torch.nn as nn
import torch.optim as optim
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm

def imgtransform():
  transform = A.Compose(
      [A.Resize(height =image_height,width=image_width),
       A.Rotate(limit=35,p=1.0),  #0~35의 randn에서 angle 결정, 100%확률 적용
       A.HorizontalFlip(p=0.5), #50%확률로 가로 뒤집기
       A.VerticalFlip(p=0.1),
       A.Normalize(mean=[0.0,0.0,0.0],#이미지 데이터 정규화
                   std=[1.0,1.0,1.0],
                   max_pixel_value=255.0),ToTensorV2()#이미지를 tensor화시킴
      ])

In [46]:
from torch.utils.data import DataLoader,Dataset
class jigsawdataset(Dataset):
  def __init__(self,imagedir,transform=None):
    self.imagedir = imagedir
    self.transform = transform
    self.images = os.listdir(imagedir)

  def __len__(self):
    return len(self.images)

  def __getitem__(self,idx):
    #dataloader가 데이터 로드할떄 __getitem__호출하며 idx가 내부적으로 제공


In [ ]:
def get_loaders
()

In [ ]:
def calc_puzzle(answer_df, submission_df):
    # Check for missing values in submission_df
    if submission_df.isnull().values.any():
        raise ValueError("The submission dataframe contains missing values.")

    # Public or Private answer Sample and Sorting by 'ID'
    submission_df = submission_df[submission_df.iloc[:, 0].isin(answer_df.iloc[:, 0])]
    submission_df = submission_df.sort_values(by='ID').reset_index(drop=True)

    # Check for length in submission_df
    if len(submission_df) != len(answer_df):
        raise ValueError("The submission dataframe wrong length.")

    # Convert position data to numpy arrays for efficient computation
    answer_positions = answer_df.iloc[:, 2:].to_numpy()  # Excluding ID, img_path, and type columns
    submission_positions = submission_df.iloc[:, 1:].to_numpy()  # Excluding ID column

    # Initialize the dictionary to hold accuracies
    accuracies = {}

    # Define combinations for 2x2 and 3x3 puzzles
    combinations_2x2 = [(i, j) for i in range(3) for j in range(3)]
    combinations_3x3 = [(i, j) for i in range(2) for j in range(2)]

    # 1x1 Puzzle Accuracy
    accuracies['1x1'] = np.mean(answer_positions == submission_positions)

    # Calculate accuracies for 2x2, 3x3, and 4x4 puzzles
    for size in range(2, 5):  # Loop through sizes 2, 3, 4
        correct_count = 0  # Initialize counter for correct full sub-puzzles
        total_subpuzzles = 0

        # Iterate through each sample's puzzle
        for i in range(len(answer_df)):
            puzzle_a = answer_positions[i].reshape(4, 4)
            puzzle_s = submission_positions[i].reshape(4, 4)
            combinations = combinations_2x2 if size == 2 else combinations_3x3 if size == 3 else [(0, 0)]

            # Calculate the number of correct sub-puzzles for this size within a 4x4
            for start_row, start_col in combinations:
                rows = slice(start_row, start_row + size)
                cols = slice(start_col, start_col + size)
                if np.array_equal(puzzle_a[rows, cols], puzzle_s[rows, cols]):
                    correct_count += 1
                total_subpuzzles += 1

        accuracies[f'{size}x{size}'] = correct_count / total_subpuzzles

    score = (accuracies['1x1'] + accuracies['2x2'] + accuracies['3x3'] + accuracies['4x4']) / 4.
    return score

In [ ]:
import PIL
from torchvision import datasets, transforms
t=[]
t.append(transforms.Resize(size, interpolation=PIL.Image.BICUBIC))

In [ ]:
# Copyright (c) Meta Platforms, Inc. and affiliates.
# All rights reserved.

# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
# --------------------------------------------------------
# References:
# timm: https://github.com/rwightman/pytorch-image-models/tree/master/timm
# DeiT: https://github.com/facebookresearch/deit
# --------------------------------------------------------


In [45]:
shuffle_order = row[[str(i) for i in range(1, 17)]].values-1
shuffle_order

NameError: name 'row' is not defined